In [265]:
class Leaf:
    def __init__(self,origin,left,right=''):
        self.origin = origin
        if len(left) == 0:
            self.left = '$'
            self.right = 1
        else:
            self.left = left
            self.right = Leaf(None,right)
    
    def __repr__(self): return '(' + repr(self.left) + ',' + repr(self.right) + ')'

    def __eq__(self,other): return self[0] == other[0]

    def __hash__(self): return hash(self[0])

    def __getitem__(self,index): return self.left[index]

    def __len__(self):
        if self.left == '$': return 0
        return len(self.left)+len(self.right)

In [266]:
class Branch:
    def __init__(self):
        self.b = {}

    def __repr__(self): 
        s = ''
        for a in self.b.values(): 
            s += repr(a) 
        return s

    def __str__(self):
        s = '' 
        for i in range(len(list(self.b.values()))-1):s+=str(list(self.b.values())[i])+'\n'
        return s+str(list(self.b.values())[-1])
    
    def __getitem__(self,index): return self.b[index]
    
    def __contains__(self,other): return other in self.b

    def __len__(self): return len(self.b)

    def is_shallow(self):
        for x in self.b.values():
            if type(x.right) == Leaf or x.left == '$': continue
            else: 
                for y in x.right.b.values():
                    if type(y.right) == Leaf or y.left == '$': continue
                    else: return False
        return True

    def add(self,leaf):
        if leaf in self.b:
            if self.b[leaf].left == '$':
                self.b[leaf].right+=1
                return
            i = 0
            left = ''
            while i < min(len(leaf.left),len(self.b[leaf].left)) and leaf.left[i] == self.b[leaf][i]: 
                left += leaf.left[i]
                i += 1
            if type(self.b[leaf].right) == Branch:
                nleaf = Leaf(leaf.origin,leaf.left[i:])
                if len(self.b[leaf].left[i:]):
                    branch = Branch()
                    l = Leaf(self.b[leaf].origin,self.b[leaf].left[i:])
                    l.right = self.b[leaf].right
                    branch.b[l] = l
                    self.b[leaf].right = branch
                self.b[leaf].right.add(nleaf)
            else: 
                branch = Branch()
                branch.add(Leaf(self.b[leaf].origin,self.b[leaf].left[i:]))
                branch.add(Leaf(leaf.origin,leaf.left[i:]))
                self.b[leaf].right = branch
            self.b[leaf].left = left
        else: 
            self.b[leaf] = leaf

In [287]:
class Sequitur:
    def __init__(self,reads):
        self.branch = Branch()
        self.reads = reads
        for read in reads: 
            for i in range(len(read)): self.branch.add(Leaf(read,read[i:]))
        self.sequence = ''

    def sequitur(self,seq,seqs,stops=None,initial=None,terminal=None):
        i = 1
        q = seq[-i:]
        branch = self.branch.b[q[0]]
        options = []
        while i <= len(seq):
            q = q[len(branch.left):]
            while len(q) > 0:
                branch = branch.right[q[0]]
                q = q[len(branch.left):]
            i += 1
            q = seq[-i:]
            if stops:
                if stops[0].startswith(q):
                    i_ = 0
                    q_ = stops[0][len(q)+i_]  
                    br = branch.right[q_]
                    while type(br.right) == Branch:
                        i_ += len(br.left)
                        q_ = stops[0][len(q)+i_]
                        br = br.right[q_]
                    if stops[0].startswith(br.origin): terminal = (br,q)
            if type(branch.right) == Leaf or branch.right.is_shallow(): break
            branch = self.branch.b[q[0]]
        if not initial:
            if type(branch.right) == Leaf: options += [branch]
            else:
                for a in branch.right.b:
                    if a.left != '$': options += [a]
        else: options = initial
        options_ = []
        for o in options:
            if type(o.right) == Branch:
                for r in o.right.b.values(): 
                    if r.left == '$': continue
                    l = Leaf(r.origin,o.left+r.left)
                    options += [l]
                continue
            if stops and o.origin in stops[1]:
                if seq.endswith(stops[0][:stops[0].find(q)+len(q)]): 
                    seq += stops[0][stops[0].find(q)+len(q):]
                    seqs.update(stops[1])
                    options += stops[2]
                    return seq, seqs, options, False, terminal
            if seq.endswith(o.origin[:o.origin.find(q)+len(q)]) and o.origin not in seqs: options_ += [o]
        options = options_
        if stops is not None and len(options) == 0 and terminal is not None:
              if seq.endswith(terminal[1]) and stops[0].startswith(terminal[1]): 
                seq += stops[0][len(terminal[1]):]
                seqs.update(stops[1])
                options += stops[2]
                return seq, seqs, options, False, None
        extension = False
        if len(options) == 1: 
            o = options.pop()
            seqs.add(o.origin)
            seq += o.left
            extension = True
        return seq, seqs, options, extension, terminal

    def assemble(self,start=None,seed=None):
        import random
        if seed is not None: random.seed(seed)

        if start is None or start not in self.self.reads: start = random.choice(self.reads)
        print('Initus:',start)
        self.sequence = self.sequitur(start,{start})
        while len(set(self.reads).difference(self.sequence[1])) > 0: 
            self.sequence = self.sequitur(self.sequence[0],self.sequence[1],terminal=self.sequence[4])
            if len(self.sequence[2]):
                nseq = random.choice(self.sequence[2]).origin
                nseq = self.sequitur(nseq,{nseq},stops=self.sequence,terminal=self.sequence[4])
                while nseq[3] and len(nseq[2]) == 0: nseq = self.sequitur(nseq[0],nseq[1],stops=self.sequence)
                self.sequence = self.sequitur(nseq[0],nseq[1],initial=nseq[2],terminal=self.sequence[4])
            if not self.sequence[3] and len(self.sequence[2]) == 0 and len(set(self.reads).difference(self.sequence[1])):
                nseq = random.choice((list(set(self.reads).difference(self.sequence[1]))))
                while nseq in self.sequence[0] and len(set(self.reads).difference(self.sequence[1])): 
                    self.sequence[1].add(nseq)
                    if not len(set(self.reads).difference(self.sequence[1])): continue
                    nseq = random.choice((list(set(self.reads).difference(self.sequence[1]))))
                if nseq not in self.sequence[0]:
                    nseq = self.sequitur(nseq,{nseq},stops=self.sequence,terminal=self.sequence[4])
                    while nseq[3] and len(nseq[2]) == 0: nseq = self.sequitur(nseq[0],nseq[1],stops=self.sequence,terminal=self.sequence[4])
                    self.sequence = self.sequitur(nseq[0],nseq[1],initial=nseq[2],terminal=self.sequence[4])

In [292]:
import random

n = 2000
seeds = [random.randint(0,13350) for a in range(n)]
seeds = list(set(seeds))
n = len(seeds)

correct_sequence = 'betty_bought_butter_the_butter_was_bitter_betty_bought_better_butter_to_make_the_bitter_butter_better'
print('Sequence:',correct_sequence)

successes = 0
reads = ['betty_bought_butter_th','tter_the_butter_was_','he_butter_was_bitter_','as_bitter_betty_bought','tty_bought_better_butter_t','r_butter_to_make_the_','ke_the_bitter_butter_better']
for i in range(n):
    random.seed(seeds[i])
    random.shuffle(reads)

    seq = Sequitur(reads)
    seq.assemble(seed=seeds[i])

    print('Seed:',seeds[i])
    if seq.sequence[0] == correct_sequence: 
        print('[SUCCESS]')
        successes+=1
    else: print('[FAILURE]')
    print('------------------------')
print('Accuracy:',successes/n*100,'%')
print('========================')

Sequence: betty_bought_butter_the_butter_was_bitter_betty_bought_better_butter_to_make_the_bitter_butter_better
Initus: as_bitter_betty_bought
Seed: 8194
[SUCCESS]
------------------------
Initus: betty_bought_butter_th
Seed: 8206
[SUCCESS]
------------------------
Initus: tter_the_butter_was_
Seed: 22
[SUCCESS]
------------------------
Initus: as_bitter_betty_bought
Seed: 8214
[SUCCESS]
------------------------
Initus: tter_the_butter_was_
Seed: 8217
[SUCCESS]
------------------------
Initus: as_bitter_betty_bought
Seed: 26
[SUCCESS]
------------------------
Initus: tter_the_butter_was_
Seed: 25
[SUCCESS]
------------------------
Initus: he_butter_was_bitter_
Seed: 31
[SUCCESS]
------------------------
Initus: tter_the_butter_was_
Seed: 46
[SUCCESS]
------------------------
Initus: ke_the_bitter_butter_better
Seed: 8243
[SUCCESS]
------------------------
Initus: he_butter_was_bitter_
Seed: 8245
[SUCCESS]
------------------------
Initus: he_butter_was_bitter_
Seed: 55
[SUCCESS]
-------

In [293]:
import random

n = 2000
seeds = [random.randint(0,13350) for a in range(n)]
seeds = list(set(seeds))
n = len(seeds)

correct_sequence = 'she_sells_sea_shells_on_the_sea_shore'
print('Sequence:',correct_sequence)

successes = 0
reads = ['she_sells_s',
               'lls_sea_shel',
                    'ea_shells_o',
                       'shells_on_the_s',
                                  'he_sea_s',
                                      'ea_shore']
for i in range(n):
    random.seed(seeds[i])
    random.shuffle(reads)

    seq = Sequitur(reads)
    seq.assemble(seed=seeds[i])

    print('Seed:',seeds[i])
    if seq.sequence[0] == correct_sequence: 
        print('[SUCCESS]')
        successes+=1
    else: print('[FAILURE]')
    print('------------------------')
print('Accuracy:',successes/n*100,'%')
print('========================')

Sequence: she_sells_sea_shells_on_the_sea_shore
Initus: ea_shore
Seed: 8195
[SUCCESS]
------------------------
Initus: shells_on_the_s
Seed: 8198
[SUCCESS]
------------------------
Initus: shells_on_the_s
Seed: 6
[SUCCESS]
------------------------
Initus: shells_on_the_s
Seed: 8200
[SUCCESS]
------------------------
Initus: ea_shore
Seed: 8201
[SUCCESS]
------------------------
Initus: shells_on_the_s
Seed: 8204
[SUCCESS]
------------------------
Initus: she_sells_s
Seed: 8212
[SUCCESS]
------------------------
Initus: ea_shore
Seed: 22
[SUCCESS]
------------------------
Initus: shells_on_the_s
Seed: 23
[SUCCESS]
------------------------
Initus: he_sea_s
Seed: 8216
[SUCCESS]
------------------------
Initus: ea_shore
Seed: 25
[SUCCESS]
------------------------
Initus: shells_on_the_s
Seed: 8218
[SUCCESS]
------------------------
Initus: shells_on_the_s
Seed: 35
[SUCCESS]
------------------------
Initus: she_sells_s
Seed: 38
[SUCCESS]
------------------------
Initus: shells_on_the_s
Seed